In [2]:
import sys
import os
import pandas as pd

path = os.path.abspath('../..')
if path not in sys.path:
    sys.path.insert(0, path)
    
sys.path

from primer3plus.utils import reverse_complement as rc
import primer3
from aqbt.contrib.uwbf import primer_utils
from aqbt import AquariumBuildTools
aqtools = AquariumBuildTools.from_toml('creds.secret.toml')
aqtools.sessions

aq = aqtools.sessions['production']['aquarium']

# Utils

In [139]:
from pydent.base import ModelBase
from pydent import Planner
from tqdm.auto import tqdm 
from pydent import Planner
from itertools import zip_longest
from typing import *
from pydent.models import Plan
from matplotlib import pylab as plt


def get_sample(session, x, *args, **kwargs):
    if isinstance(x, int):
        return session.Sample.find(x, *args, **kwargs)
    elif isinstance(x, str):
        return session.Sample.find_by_name(x, *args, **kwargs)
    elif isinstance(x, dict):
        return session.Sample.where(x, *args, **kwargs)
    elif issubclass(x.__class__, ModelBase):
        return x
    

def from_template(template):
    template = Planner(template.session, plan_id=template.id)
    planner = template.copy()
    planner.plan.status = 'planning'
    return planner
    
def grouper(iterable, n, fillvalue=...):
    args = [iter(iterable)] * n
    for x in zip_longest(*args, fillvalue=fillvalue):
        if fillvalue is not ...:
            yield x
        else:
            yield tuple([_x for _x in x if _x is not ...])


def from_template(template):
    template = Planner(template)
    planner = template.copy()
    planner.plan.status = 'planning'
    return planner

def combine(plans: List[Plan]) -> "Planner":
    """Merges a list of plans into a single plan by combining operations
    and wires.

    :param plans: list of Aquarium Plans instances
    :return: new Plan
    """
    copied_plans = [c.copy() for c in plans]

    sessions = {p.session for p in plans}
#     if len(sessions) > 1:
#         raise PlannerException(
#             "Cannot combine plans, plans must all derive from same session instance"
#         )
    session = sessions.pop()

    new_plan = Planner(session)
    new_plan.plan.operations = []
    for p in copied_plans:
        new_plan.plan.operations += p.plan.operations
        new_plan.plan.wires += p.plan.wires
    return new_plan


def submit_oe(session, sample):
    template = session.Plan.where({'name': '(SD2) Overlap Extension PCR Template', 'status': 'template'})[0]

    planner = from_template(template)

    op = planner.get_op_by_name('Stitch by Overlap Extension')[0]
    op.set_field_value_array('Fragment Mix', 'input', [{'sample': s} for s in sample.properties['Fragment Mix Array']])
    planner.set_field_value_and_propogate(op.outputs[0], sample=sample)
    op = planner.get_op_by_name('Make PCR Fragment')[0]
    planner.set_field_value_and_propogate(op.outputs[0], sample=sample)
    planner.set_inputs_using_sample_properties(op, sample=sample)
    planner.set_field_value(op.input('Template'), object_type=aq.ObjectType.find_by_name('Unverified Fragment PCR Fragment'))
    planner.plan.name = 'OE Ext {}'.format(sample.name)
    return planner

# Load Data

In [4]:
import json 

with open('build_permutations_2021-03-08T10:16:09.773080-08:00.json', 'r') as f:
    builds_json = json.load(f)

## Submit Alpha Strains

In [55]:
transformations = set()

for circuit in builds_json:
    for part in circuit['parts']:
        if 'yeGFP' in part['sample']['name']:
            transformations.add((part['haploid'], part['sample']['name']))
            
for _, name in transformations:
    frag = aq.Sample.find_by_name(name)
    frag_stock = aq.Item.where({'sample_id': frag.id, 'object_type_id': aq.ObjectType.find_by_name('Fragment Stock').id})
    print(name)

URA-URA<pGRR-W10W20-yeGFP>
URA-URA<pGRR-W17W34-yeGFP>
URA-URA<pGRR-W17W17-yeGFP>
URA-URA<pGRR-W10W17-yeGFP>
URA-URA<pGRR-W20W36-yeGFP>
URA-URA<pGRR-W5W36-yeGFP>
URA-URA<pGRR-W36W36-yeGFP>
URA-URA<pGRR-W36W5-yeGFP>


In [95]:
def get_gfp_part(circuit):
    return [part for part in circuit['parts'] if 'gfp' in part['sample']['name'].lower()][0]

def get_galz4_part(circuit):
    return [part for part in circuit['parts'] if 'galz4' in part['sample']['name'].lower()][0]

get_gfp_part(builds_json[0])
get_galz4_part(builds_json[0])

def create_transformation(circuit):
    base_name = 'CEN.PK2 - MAT alpha ::: pMODKan-HO-pACT1-ZEV4 | ' + get_galz4_part(circuit)['sample']['name']
    parent = aq.Sample.find_by_name(base_name)
    
    integrant = aq.Sample.find_by_name(get_gfp_part(circuit)['sample']['name'])
    return base_name, {'parent': parent, 'integrant': integrant}
    
create_transformation(builds_json[0])

def create_new_strain(parent, integrant, qc1, qc2, qcl):
    parent = get_sample(aq, parent)
    integrant = get_sample(aq, integrant)

    properties = {
        'Parent': parent,
        'Integrant': integrant,
        'Plasmid': None,
        'Integrated Marker(s)': integrant.properties['Yeast Marker'],
        'Mating Type': parent.properties['Mating Type'],
        'Comp_cell_limit': 'No',
        'Has this strain passed QC?': 'No',
        'Haploids': [],
        'QC Primer1': get_sample(aq, qc1),
        'QC Primer2': get_sample(aq, qc2),
        'QC_length': qcl
    }

    return aq.Sample.new(
        name = parent.name + ' | ' + integrant.name,
        description='',
        sample_type_id=parent.sample_type_id,
        project='SD2 2021 Redesigns',
        properties=properties
    )

new_strains = dict([create_transformation(c) for c in builds_json])

new_samples = []
for x in tqdm(list(new_strains.values())):
    sample = create_new_strain(x['parent'], x['integrant'], 'ColonyPCR_URA3_F(pMOD)', 'PS_Rev', 798)
    existing = aq.Sample.find_by_name(sample.name)
    if existing is None:
        new_samples.append(sample)
        
for s in tqdm(new_samples):
    s.save()
    

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

In [108]:
for x in new_samples:
    s = aq.Sample.find_by_name(x.name)
    print(s.name)

CEN.PK2 - MAT alpha ::: pMODKan-HO-pACT1-ZEV4 | pMOD8-pGALZ4-URGR-W10 | URA-URA<pGRR-W5W36-yeGFP>
CEN.PK2 - MAT alpha ::: pMODKan-HO-pACT1-ZEV4 | pMOD8-pGALZ4-URGR-W8 | URA-URA<pGRR-W17W17-yeGFP>
CEN.PK2 - MAT alpha ::: pMODKan-HO-pACT1-ZEV4 | pMOD8-pGALZ4-URGR-W36 | URA-URA<pGRR-W36W5-yeGFP>
CEN.PK2 - MAT alpha ::: pMODKan-HO-pACT1-ZEV4 | pMOD8-pGALZ4-URGR-W17 | URA-URA<pGRR-W17W34-yeGFP>
CEN.PK2 - MAT alpha ::: pMODKan-HO-pACT1-ZEV4 | pMOD8-pGALZ4-URGR-W5 | URA-URA<pGRR-W36W36-yeGFP>


In [124]:
from pydent import Planner

for sample in tqdm(new_samples):
    planner = Planner(aq)

    planner.chain(
        'Yeast Transformation', 
        'Check Yeast Plate', 
        'Yeast Overnight Suspension', 
        'Inoculate Yeast Comp Cells', 
        'Make Yeast Comp Cells'
    )
    op = planner.get_op_by_name('Yeast Overnight Suspension')[0]
    planner.set_field_value(op.input('Type of Media'), value='YPAD')
    planner.set_field_value(op.input('Require QC?'), value='yes')
    planner.chain(op, 'Yeast Glycerol Stock')

    op = planner.get_op_by_name('Check Yeast Plate')[0]
    for i in range(4):
        planner.chain(op, 'Yeast Lysate', 'Colony PCR', 'Fragment Analyzing')

    op = planner.get_op_by_name('Yeast Transformation')[0]
    planner.set_field_value_and_propogate(op.outputs[0], sample=sample)
    planner.set_inputs_using_sample_properties(op, sample=sample)
    planner.set_field_value(op.input('Genetic Material'), sample=sample.properties['Integrant'])
    for op in planner.get_op_by_name('Colony PCR'):
        planner.set_inputs_using_sample_properties(op, sample=sample)

    planner.prettify()
    planner.name = 'Transform {}'.format(sample.name)
    planner.save()
    
    print(planner.url)

  0%|          | 0/5 [00:00<?, ?it/s]

http:/52.27.43.242/plans?plan_id=40842
http:/52.27.43.242/plans?plan_id=40843
http:/52.27.43.242/plans?plan_id=40844
http:/52.27.43.242/plans?plan_id=40845
http:/52.27.43.242/plans?plan_id=40846


# Submit A Strains

In [146]:
integrants = set()

for circuit in builds_json:
    parts = circuit['parts']
    mataparts = [part for part in parts if part['haploid'] == 'Mat A']
    integrant = [x['sample']['name'] for x in mataparts if 'teto' in x['sample']['name'].lower()][0]
    integrants.add(integrant)
integrants

{'HIS-HIS<7XTetO-pMinCyc1-URGR-W34>',
 'HIS-HIS<7XTetO-pMinCyc1-URGR-W5>',
 'HIS-HIS<7XTetO-pMinCyc1-URGR-W8>'}

In [150]:
base_strains = [
    'CEN.PK2 - MAT A ::: pMOD4G-dcas9-mxi1',
    'CEN.PK2 - MAT A ::: pMOD4G-dcas9-mxi1 | pMOD-HO-KanMX-pTDH3-rtTA-VP16',
    'CEN.PK2 - MAT A ::: pMOD4G-dcas9-mxi1 | pMOD-HO-KanMX-pTDH3-rtTA-VPR',
    'CEN.PK2 - MAT A ::: pMOD4G-dcas9-mxi1 | pMOD-HO-KanMX-pREV1-rtTA-VP16',
    'CEN.PK2 - MAT A ::: pMOD4G-dcas9-mxi1 | pMOD-HO-KanMX-pREV1-rtTA-VPR',
    'CEN.PK2 - MAT A ::: pMOD4G-dcas9-mxi1 | pMOD-HO-KanMX-pRPL18B-rtTA-VP16'
]

new_strains = []
for base in base_strains:
    parent = aq.Sample.find_by_name(base)
    assert parent
    for integrant in integrants:
        new_strains.append(create_new_strain(parent, integrant, qc1='ColonyPCR_HIS3_F(pMOD)', qc2='PS_Rev', qcl=2837))

In [153]:
for sample in new_strains:
    existing = aq.Sample.find_by_name(sample.name)
    if existing:
        print('{} exists'.format(sample.name))
    else:
        sample.save()

In [155]:
from pydent import Planner

for sample in tqdm(new_strains):
    planner = Planner(aq)

    planner.chain(
        'Yeast Transformation', 
        'Check Yeast Plate', 
        'Yeast Overnight Suspension', 
        'Inoculate Yeast Comp Cells', 
        'Make Yeast Comp Cells'
    )
    op = planner.get_op_by_name('Yeast Overnight Suspension')[0]
    planner.set_field_value(op.input('Type of Media'), value='YPAD')
    planner.set_field_value(op.input('Require QC?'), value='yes')
    planner.chain(op, 'Yeast Glycerol Stock')

    op = planner.get_op_by_name('Check Yeast Plate')[0]
    for i in range(4):
        planner.chain(op, 'Yeast Lysate', 'Colony PCR', 'Fragment Analyzing')

    op = planner.get_op_by_name('Yeast Transformation')[0]
    planner.set_field_value_and_propogate(op.outputs[0], sample=sample)
    planner.set_inputs_using_sample_properties(op, sample=sample)
    planner.set_field_value(op.input('Genetic Material'), sample=sample.properties['Integrant'])
    for op in planner.get_op_by_name('Colony PCR'):
        planner.set_inputs_using_sample_properties(op, sample=sample)

    planner.prettify()
    planner.name = 'Transform {}'.format(sample.name)
    planner.save()
    
    print(planner.url)

  0%|          | 0/18 [00:00<?, ?it/s]

http:/52.27.43.242/plans?plan_id=40848
http:/52.27.43.242/plans?plan_id=40849
http:/52.27.43.242/plans?plan_id=40850
http:/52.27.43.242/plans?plan_id=40851
http:/52.27.43.242/plans?plan_id=40852
http:/52.27.43.242/plans?plan_id=40853
http:/52.27.43.242/plans?plan_id=40854
http:/52.27.43.242/plans?plan_id=40855
http:/52.27.43.242/plans?plan_id=40856
http:/52.27.43.242/plans?plan_id=40857
http:/52.27.43.242/plans?plan_id=40858


Future exception was never retrieved
future: <Future finished exception=KeyError(18627)>
Traceback (most recent call last):
  File "/home/justin/anaconda3/envs/aqbt/lib/python3.8/concurrent/futures/thread.py", line 57, in run
    result = self.fn(*self.args, **self.kwargs)
  File "/home/justin/anaconda3/envs/aqbt/lib/python3.8/site-packages/pydent/utils/async_requests.py", line 20, in wrapped
    return (i, fxn(*args, **kwargs))
  File "/home/justin/anaconda3/envs/aqbt/lib/python3.8/site-packages/pydent/planner/graph.py", line 70, in add_wires
    raise e
  File "/home/justin/anaconda3/envs/aqbt/lib/python3.8/site-packages/pydent/planner/graph.py", line 66, in add_wires
    from_id = _id_getter(fv_to_op_dict[wire.source.rid])
KeyError: 18627


http:/52.27.43.242/plans?plan_id=40859
http:/52.27.43.242/plans?plan_id=40860
http:/52.27.43.242/plans?plan_id=40861
http:/52.27.43.242/plans?plan_id=40862
http:/52.27.43.242/plans?plan_id=40863
http:/52.27.43.242/plans?plan_id=40864
http:/52.27.43.242/plans?plan_id=40866
